In [2]:
!pip install -Uq catalyst gym

     |████████████████████████████████| 543 kB 3.8 MB/s 
     |████████████████████████████████| 1.6 MB 34.9 MB/s 
     |████████████████████████████████| 636 kB 45.4 MB/s 
     |████████████████████████████████| 120 kB 65.7 MB/s 


# Seminar. RL, DQN.

Hi! In the first part of the seminar, we are going to introduce one of the main algorithm in the Reinforcment Learning domain. Deep Q-Network is the pioneer algorithm, that amalmagates Q-Learning and Deep Neural Networks. And there is small review on gym enviroments, where our bots will play in games.

In [1]:
from collections import deque, namedtuple
import random
import numpy as np
import gym

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from catalyst import dl, utils

In the beginning, look at the algorithm:

![DQN algorithm](https://i.stack.imgur.com/Jnyff.jpg)
There are several differences between the usual DL and RL routines. Our bots are trained by his actions, that he has done in the past. We don't have infinity memory, but we can save some actions in the buffer. Let's code it!

In [2]:
device = utils.get_device()

In [5]:
import numpy as np
from collections import deque, namedtuple

Transition = namedtuple(
    'Transition', 
    field_names=[
        'state', 
        'action', 
        'reward',
        'done', 
        'next_state'
    ]
)

class ReplayBuffer:
    def __init__(self, capacity: int):
        self.buffer = deque(maxlen=capacity)
    
    def append(self, transition: Transition):
        self.buffer.append(transition)
    
    def sample(self, size: int):
        indices = np.random.choice(
            len(self.buffer), 
            size, 
            replace=size > len(self.buffer)
        )
        states, actions, rewards, dones, next_states = \
            zip(*[self.buffer[idx] for idx in indices])
        states, actions, rewards, dones, next_states = (
            np.array(states, dtype=np.float32), 
            np.array(actions, dtype=np.int64), 
            np.array(rewards, dtype=np.float32),
            np.array(dones, dtype=np.bool), 
            np.array(next_states, dtype=np.float32)
        )
        return states, actions, rewards, dones, next_states
    
    def __len__(self):
        return len(self.buffer)

To work well with Catalyst train loops, implement intermedeate abstraction.

In [6]:
from torch.utils.data.dataset import IterableDataset

# as far as RL does not have some predefined dataset, 
# we need to specify epoch lenght by ourselfs
class ReplayDataset(IterableDataset):
    def __init__(self, buffer: ReplayBuffer, epoch_size: int = int(1e3)):
        self.buffer = buffer
        self.epoch_size = epoch_size

    def __iter__(self):
        states, actions, rewards, dones, next_states = \
            self.buffer.sample(self.epoch_size)
        for i in range(len(dones)):
            yield states[i], actions[i], rewards[i], dones[i], next_states[i]
    
    def __len__(self):
        return self.epoch_size

After creating a Buffer, we need to gather action-value-state and save it in the buffer. We create one function, that asks model for action, and another function to communicate with the enviroment.

In [8]:
def get_action(env, network, state, epsilon=-1):
    if np.random.random() < epsilon:
        action = env.action_space.sample()
    else:
        state = torch.tensor(state[None], dtype=torch.float32).to(device)
        q_values = network(state).detach().cpu().numpy()[0]
        action = np.argmax(q_values)

    return int(action)


def generate_session(
    env, 
    network, 
    t_max=1000, 
    epsilon=-1,
    replay_buffer=None,
):
    total_reward = 0
    state = env.reset()

    for t in range(t_max):
        action = get_action(env, network, state=state, epsilon=epsilon)
        next_state, reward, done, _ = env.step(action)

        if replay_buffer is not None:
            transition = Transition(
                state, action, reward, done, next_state)
            replay_buffer.append(transition)

        total_reward += reward
        state = next_state
        if done:
            break

    return total_reward, t

def generate_sessions(
    env, 
    network, 
    t_max=1000, 
    epsilon=-1,
    replay_buffer=None,
    num_sessions=100,
):
    sessions_reward, sessions_steps = 0, 0
    for i_episone in range(num_sessions):
        r, t = generate_session(
            env=env, 
            network=network,
            t_max=t_max,
            epsilon=epsilon,
            replay_buffer=replay_buffer,
        )
        sessions_reward += r
        sessions_steps += t
    return sessions_reward, sessions_steps

If we look closely into algorithm, we'll see that we need two networks. They looks the same, but one updates weights by gradients algorithm and second one by moving average with the first. This process helps to get stable training by REINFORCE.

In [9]:
def soft_update(target, source, tau):
    """Updates the target data with smoothing by ``tau``"""
    for target_param, param in zip(target.parameters(), source.parameters()):
        target_param.data.copy_(
            target_param.data * (1.0 - tau) + param.data * tau
        )

To communicate with the Buffer, Catalyst's Runner requires adiitional Callback.

In [9]:
class GameCallback(dl.Callback):
    
    def __init__(
        self, 
        *, 
        env, 
        replay_buffer, 
        session_period, 
        epsilon,
        epsilon_k,
        actor_key,
    ):
        super().__init__(order=0)
        self.env = env
        self.replay_buffer = replay_buffer
        self.session_period = session_period
        self.epsilon = epsilon
        self.epsilon_k = epsilon_k
        self.actor_key = actor_key
    
    def on_stage_start(self, runner: dl.IRunner):
        self.actor = runner.model[self.actor_key]
        
        self.actor.eval()
        generate_sessions(
            env=self.env, 
            network=self.actor,
            epsilon=self.epsilon,
            replay_buffer=self.replay_buffer,
            num_sessions=1000,
        )
        self.actor.train()

    def on_epoch_start(self, runner: dl.IRunner):
        self.epsilon *= self.epsilon_k
        self.session_counter = 0
        self.session_steps = 0
    
    def on_batch_end(self, runner: dl.IRunner):
        if runner.global_batch_step % self.session_period == 0:
            self.actor.eval()
            
            session_reward, session_steps = generate_session(
                env=self.env, 
                network=self.actor,
                epsilon=self.epsilon,
                replay_buffer=self.replay_buffer
            )

            self.session_counter += 1
            self.session_steps += session_steps

            runner.batch_metrics.update({"s_reward": session_reward})
            runner.batch_metrics.update({"s_steps": session_steps})
            
            self.actor.train()

    def on_epoch_end(self, runner: dl.IRunner):
        num_sessions = 100
        
        self.actor.eval()
        valid_rewards, valid_steps = generate_sessions(
            env=self.env, 
            network=self.actor,
            num_sessions=num_sessions
        )
        self.actor.train()
        
        valid_rewards /= float(num_sessions)
        valid_steps /= float(num_sessions)
        runner.epoch_metrics["_epoch_"]["num_samples"] = self.session_steps
        runner.epoch_metrics["_epoch_"]["updates_per_sample"] = (
            runner.loader_sample_step / self.session_steps
        )
        runner.epoch_metrics["_epoch_"]["v_reward"] = valid_rewards

In [10]:
class CustomRunner(dl.Runner):
    
    def __init__(
        self, 
        *, 
        gamma, 
        tau, 
        tau_period=1, 
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.gamma = gamma
        self.tau = tau
        self.tau_period = tau_period
    
    def on_stage_start(self, runner: dl.IRunner):
        super().on_stage_start(runner)
        soft_update(self.model["target"], self.model["origin"], 1.0)

    def handle_batch(self, batch):
        # model train/valid step
        states, actions, rewards, dones, next_states = batch
        network, target_network = self.model["origin"], self.model["target"]

        # get q-values for all actions in current states
        state_qvalues = network(states)
        # select q-values for chosen actions
        state_action_qvalues = \
            state_qvalues.gather(1, actions.unsqueeze(-1)).squeeze(-1)
        
        # compute q-values for all actions in next states
        # compute V*(next_states) using predicted next q-values
        # at the last state we shall use simplified formula: 
        # Q(s,a) = r(s,a) since s' doesn't exist
        with torch.no_grad():
            next_state_qvalues = target_network(next_states)
            next_state_values = next_state_qvalues.max(1)[0]
            next_state_values[dones] = 0.0
            next_state_values = next_state_values.detach()

        # compute "target q-values" for loss, 
        # it's what's inside square parentheses in the above formula.
        target_state_action_qvalues = \
            next_state_values * self.gamma + rewards

        # mean squared error loss to minimize
        loss = self.criterion(
            state_action_qvalues,
            target_state_action_qvalues.detach()
        )
        self.batch_metrics.update({"loss": loss})

        if self.is_train_loader:
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()

            if self.global_batch_step % self.tau_period == 0:
                soft_update(target_network, network, self.tau)

In [11]:
def get_network(env, num_hidden=128):
    inner_fn = utils.get_optimal_inner_init(nn.ReLU)
    outer_fn = utils.outer_init
    
    network = torch.nn.Sequential(
        nn.Linear(env.observation_space.shape[0], num_hidden),
        nn.ReLU(),
        nn.Linear(num_hidden, num_hidden),
        nn.ReLU(),
    )
    head = nn.Linear(num_hidden, env.action_space.n)
    
    network.apply(inner_fn)
    head.apply(outer_fn)

    return torch.nn.Sequential(network, head)

In [12]:
# data
batch_size = 64
epoch_size = int(1e3) * batch_size
buffer_size = int(1e5)
# runner settings, ~training
gamma = 0.99
tau = 0.01
tau_period = 1 # in batches
# callback, ~exploration
session_period = 100 # in batches
epsilon = 0.98
epsilon_k = 0.9
# optimization
lr = 3e-4

# env_name = "LunarLander-v2"
env_name = "CartPole-v1"
env = gym.make(env_name)
replay_buffer = ReplayBuffer(buffer_size)

network, target_network = get_network(env), get_network(env)
utils.set_requires_grad(target_network, requires_grad=False)

models = {"origin": network, "target": target_network}
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(network.parameters(), lr=lr)

loaders = {
    "train": DataLoader(
        ReplayDataset(replay_buffer, epoch_size=epoch_size), 
        batch_size=batch_size,
    ),
}


runner = CustomRunner(
    gamma=gamma, 
    tau=tau,
    tau_period=tau_period,
    
)

runner.train(
    model=models,
    criterion=criterion,
    optimizer=optimizer,
    loaders=loaders,
    logdir="./logs_dqn",
    num_epochs=10,
    verbose=True,
    valid_loader="_epoch_",
    valid_metric="v_reward",
    minimize_valid_metric=False,
    load_best_on_end=True,
    callbacks=[
        GameCallback(
            env=env, 
            replay_buffer=replay_buffer, 
            session_period=session_period,
            epsilon=epsilon,
            epsilon_k=epsilon_k,
            actor_key="origin",
        )
    ]
)

1/10 * Epoch (train):   0%|          | 0/1000 [00:00<?, ?it/s]

train (1/10) 
* Epoch (1/10) num_samples: 215.0 | updates_per_sample: 297.6744186046512 | v_reward: 289.74


2/10 * Epoch (train):   0%|          | 0/1000 [00:00<?, ?it/s]

train (2/10) 
* Epoch (2/10) num_samples: 467.0 | updates_per_sample: 137.04496788008566 | v_reward: 224.81


3/10 * Epoch (train):   0%|          | 0/1000 [00:00<?, ?it/s]

...

train (8/10) 
* Epoch (8/10) num_samples: 1433.0 | updates_per_sample: 44.66154919748779 | v_reward: 248.19


9/10 * Epoch (train):   0%|          | 0/1000 [00:00<?, ?it/s]

train (9/10) 
* Epoch (9/10) num_samples: 1983.0 | updates_per_sample: 32.27433182047403 | v_reward: 267.53


10/10 * Epoch (train):   0%|          | 0/1000 [00:00<?, ?it/s]

train (10/10) 
* Epoch (10/10) num_samples: 1701.0 | updates_per_sample: 37.6249265138154 | v_reward: 272.41
Top best models:
logs_dqn/checkpoints/train.1.pth	289.7400


|And we can watch how our model plays in the games!

\* to run cells below, you should update your python environment. Instruction depends on your system specification.

In [13]:
# record sessions
import gym.wrappers
env = gym.wrappers.Monitor(
    gym.make(env_name),
    directory="videos_dqn", 
    force=True)
generate_sessions(
    env=env, 
    network=runner.model["origin"],
    num_sessions=100
)
env.close()

NoSuchDisplayException: ignored

In [ ]:
# show video
from IPython.display import HTML
import os

video_names = list(
    filter(lambda s: s.endswith(".mp4"), os.listdir("./videos_dqn/")))

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format("./videos/"+video_names[-1]))  # this may or may not be _last_ video. Try other indices